# Event Ingest

In [ ]:
import csv


class Rush:
    def __init__(self, name, id):
        self.name = name
        self.id = id
        self.score = 0
        self.comments = []
        self.interactions = 0
        self.interaction_names = []

    def __repr__(self):
        return (
            self.name
            + " "
            + str(self.id)
            + " "
            + str(self.score)
            + " "
            + str(self.comments)
        )


def convert_response(response):
    dict = {
        "Strongly Agree": 4,
        "Agree": 3,
        "Neutral": 2,
        "Disagree": 1,
        "Strongly Disagree": 0,
    }
    return dict[response]


def ingest(filenames, save_as):
    rushes = []
    for filename in filenames:
        with open(filename, newline="") as f:
            reader = csv.reader(f)
            data = list(reader)

        num_rows = len(data)

        running_points = 0
        running_possible_points = 0

        # this is a weird way to iterate, basically have to iterate by col, row rather than row, col because of how the sheets look from the forms
        for col in range(2, len(data[0])):
            for row in range(len(data)):
                # name:id case, initialize
                if col % 7 == 2:
                    if row == 0:
                        header = data[0][col]
                        split_data = header.split("/")
                        name = split_data[0]
                        id = split_data[1]
                        new_rush = Rush(name, id)
                        rushes.append(new_rush)
                        current_rush = new_rush
                        running_points, running_possible_points, interactions = 0, 0, 0
                # skip comment col (1) and interact column (3)
                elif col % 7 != 1 and col % 7 != 3:
                    if row != 0:
                        response = data[row][col]
                        if response != "":
                            if col % 7 == 4:
                                current_rush.interaction_names.append(
                                    data[row][1].lower())
                            interactions += 1
                            running_points += convert_response(response)
                            running_possible_points += 4
                # we are at comment row, total points
                elif col % 7 == 1:
                    if row == 0:
                        comments = []
                        if running_possible_points > 0:
                            total_score = running_points / running_possible_points
                            current_rush.score = total_score
                        else:
                            current_rush.score = 0
                        # interactions is counted for each response so 4x too many
                        current_rush.interactions = int(interactions / 4)
                    # add all the comments
                    else:
                        if data[row][col] != "":
                            comments.append(
                                data[row][1] + ": " + data[row][col])
                        if row == num_rows - 1:
                            s = ""
                            for comment in comments:
                                s += " | " + comment
                            s = s[2:]
                            current_rush.comments = s

    rushes.sort(key=lambda r: r.score, reverse=True)

    with open(save_as, "w") as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(
            ["Name", "id", "Score", "Comments", "Interactions", "Names"])
        for rush in rushes:
            # parse names into a string for csv export
            interaction_string = ""
            for name in rush.interaction_names:
                interaction_string += ", " + name
            interaction_string = interaction_string[2:]
            csvwriter.writerow(
                [
                    rush.name,
                    rush.id,
                    rush.score,
                    rush.comments,
                    rush.interactions,
                    interaction_string,
                ]
            )


# CHANGE THESE =================================================================================
event_responses_filenames = [
    "real_data/bbq_responses_1.csv",
    "real_data/bbq_responses_2.csv",
    "real_data/bbq_responses_3.csv",
    "real_data/bbq_responses_4.csv"
]
event_results_filename = "real_data/bbq_results.csv"
# ==============================================================================================

rush_scores = ingest(event_responses_filenames, event_results_filename)


# Cut Maker

In [ ]:
import csv


class Rush:
    def __init__(self, name, id):
        self.name = name
        self.id = id
        self.cut = False
        self.scores = [0, 0, 0]
        self.interactions = [0, 0, 0]
        self.total_interactions = 0
        self.interaction_names = []
        self.db_score = 0
        self.comments = ""
        self.major = ""
        self.year = ""
        self.previously_knowns = ""
        self.clubs = ""
        self.gpa = 0.0
        self.interview_score = 0
        self.head_interviewer = ""
        self.pm_score = 0
        self.rush_score = 0
        self.rush_week_score = 0
        self.brotherhood_vote = ""
        self.email = ""

    def __repr__(self):
        return (
            self.name
            + " "
            + self.id
            + " "
            + str(self.cut)
            + " "
            + str(self.scores)
            + " "
            + str(self.interactions)
            + " "
            + str(self.db_score)
            + " "
            + str(self.interview_score)
            + " "
            + str(self.pm_score)
            + " "
            + str(self.rush_score)
            + " "
            + str(self.comments)
        )

# CHANGE THESE =================================================================================
sd_scores_filename = "real_data/sd_results.csv"
mingle_scores_filename = "real_data/bm_results.csv"

rush_profile_filename = "real_data/rush_profile_responses.csv"

cut_sheet_filename = "real_data/cut.csv" # output file
# ==============================================================================================

score_filenames = [sd_scores_filename, mingle_scores_filename]

rushes = []

# rush profile data
with open(rush_profile_filename, newline="") as f:
    reader = csv.reader(f)
    data = list(reader)

for row in data[1:]:
    found = False
    for rush in rushes:
        if rush.id == row[2].lower():
            found = True
    if not found:
        new_rush = Rush(row[1], row[2].lower())
        new_rush.major = row[5]
        new_rush.year = row[8]
        new_rush.previously_knowns = row[10]
        new_rush.clubs = row[11]
        new_rush.gpa = row[12]
        new_rush.email = row[3]
        rushes.append(new_rush)

# event data
for i in range(len(score_filenames)):
    # get csv of scores from an event
    with open(score_filenames[i], newline="") as f:
        reader = csv.reader(f)
        data = list(reader)
    # add scores to local data
    for row in data[1:]:
        found = False
        for rush in rushes:
            if rush.id == row[1].lower():
                found = True
                rush.scores[i] = float(row[2])
                rush.interactions[i] = int(row[4])
                rush.total_interactions += int(row[4])
                try:
                    list_names = row[5].split(",")
                    for name in list_names:
                        if name.strip() not in rush.interaction_names:
                            rush.interaction_names.append(name.strip())
                except:
                    None
                if row[3] != "":
                    if rush.comments == "":
                        rush.comments = row[3]
                    else:
                        rush.comments = rush.comments + " | " + row[3]
                break
        # this is the case where we have data about a rush from an event, but they haven't filled out the rush profile
        if not found:
            new_rush = Rush(row[0], row[1].lower())
            print('not found', new_rush)
            rushes.append(new_rush)
            new_rush.scores[i] = float(row[2])
            new_rush.interactions[i] = int(row[4])
            new_rush.total_interactions += int(row[4])
            try:
                list_names = row[5].split(",")
                for name in list_names:
                    if name.strip() not in new_rush.interaction_names:
                        new_rush.interaction_names.append(name.strip())
            except:
                None
            if row[3] != "":
                if new_rush.comments == "":
                    new_rush.comments = row[3]
                else:
                    new_rush.comments = new_rush.comments + " | " + row[3]

# tally up event scores
for rush in rushes:
    score_sum = 0
    interactions = 0
    for i in range(2):
        score_sum += rush.scores[i] * rush.interactions[i]
        interactions += rush.interactions[i]
    if interactions > 0:
        rush.db_score = score_sum / interactions

rushes.sort(key=lambda r: r.db_score, reverse=True)

# write score breakdown
with open(cut_sheet_filename, "w") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(
        [
            "Name",
            "id",
            "Speed Dating Score",
            "Speed Dating Interactions",
            "Mingle Score",
            "Mingle Interactions",
            "Overall Score",
            "Cut",
            "Year",
            "GPA",
            "email"
        ]
    )
    for rush in rushes:
        csv_writer.writerow(
            [
                rush.name,
                rush.id,
                rush.scores[0],
                rush.interactions[0],
                rush.scores[1],
                rush.interactions[1],
                rush.db_score,
                "F",
                rush.year,
                rush.gpa,
                rush.email
            ]
        )


# Cut Maker 2

In [ ]:
import csv


class Rush:
    def __init__(self, name, id):
        self.name = name
        self.id = id
        self.cut = False
        self.scores = [0, 0, 0]
        self.interactions = [0, 0, 0]
        self.total_interactions = 0
        self.interaction_names = []
        self.db_score = 0
        self.comments = ""
        self.major = ""
        self.year = ""
        self.previously_knowns = ""
        self.clubs = ""
        self.gpa = 0.0
        self.interview_score = 0
        self.head_interviewer = ""
        self.pm_score = 0
        self.rush_score = 0
        self.rush_week_score = 0
        self.brotherhood_vote = ""

    def __repr__(self):
        return (
            self.name
            + " "
            + self.id
            + " "
            + str(self.cut)
            + " "
            + str(self.scores)
            + " "
            + str(self.interactions)
            + " "
            + str(self.db_score)
            + " "
            + str(self.interview_score)
            + " "
            + str(self.pm_score)
            + " "
            + str(self.rush_score)
            + " "
            + str(self.comments)
        )


# CHANGE THESE =================================================================================
sd_scores_filename = "real_data/sd_results_cleaned.csv"
mingle_scores_filename = "real_data/bm_results.csv"
gn_scores_filename = "real_data/gn_results.csv"

cut_sheet_filename = "real_data/cut_sheet_cleaned.csv"

cut_sheet_2_filename = "real_data/cut_info_2.csv"  # output file
# ==============================================================================================

rush_profile_filename = "real_data/rush_profile_responses_updated_3.csv"

score_filenames = [
    sd_scores_filename,
    mingle_scores_filename,
    gn_scores_filename,
]

rushes = []

# rush profile data
with open(rush_profile_filename, newline="") as f:
    reader = csv.reader(f)
    data = list(reader)

for row in data[1:]:
    found = False
    for rush in rushes:
        if rush.id == row[2].lower():
            found = True
    if not found:
        new_rush = Rush(row[1].lower(), row[2].lower())
        new_rush.major = row[5]
        new_rush.year = row[8]
        new_rush.previously_knowns = row[10]
        new_rush.clubs = row[11]
        new_rush.gpa = row[12]
        new_rush.email = row[3]
        rushes.append(new_rush)

# event data
for i in range(3):
    # get csv of scores from an event
    with open(score_filenames[i], newline="") as f:
        reader = csv.reader(f)
        data = list(reader)
    # add scores to local data
    for row in data:
        for rush in rushes:
            if rush.id == row[1].lower():
                rush.scores[i] = float(row[2])
                rush.interactions[i] = int(row[4])
                rush.total_interactions += int(row[4])
                try:
                    list_names = row[5].split(",")
                    for name in list_names:
                        if name.strip() not in rush.interaction_names:
                            rush.interaction_names.append(name.strip())
                except:
                    None
                if row[3] != "":
                    if rush.comments == "":
                        rush.comments = row[3]
                    else:
                        rush.comments = rush.comments + " | " + row[3]
                break

# tally up event scores
for rush in rushes:
    score_sum = 0
    interactions = 0
    for i in range(3):
        score_sum += rush.scores[i] * rush.interactions[i]
        interactions += rush.interactions[i]
    if interactions > 0:
        rush.db_score = score_sum / interactions

# cut sheet data
with open(cut_sheet_filename, newline="") as f:
    reader = csv.reader(f)
    data = list(reader)

for row in data[1:]:
    if row[7] == "T":
        for rush in rushes:
            if rush.id == row[1].lower():
                rush.cut = True
                break

rushes.sort(key=lambda r: r.db_score, reverse=True)

# write score breakdown
with open(cut_sheet_2_filename, "w") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(
        [
            "Name",
            "id",
            "Speed Dating Score",
            "Speed Dating Interactions",
            "Mingle Score",
            "Mingle Interactions",
            "Game Night Score",
            "Game Night Interactions",
            "Overall Score",
            "Comments",
            "Cut",
            "email"
        ]
    )
    for rush in rushes:
        if not rush.cut:
            csv_writer.writerow(
                [
                    rush.name,
                    rush.id,
                    rush.scores[0],
                    rush.interactions[0],
                    rush.scores[1],
                    rush.interactions[1],
                    rush.scores[2],
                    rush.interactions[2],
                    rush.db_score,
                    rush.comments,
                    "F",
                    rush.email
                ]
            )


# Final Score Calculator

In [ ]:
import csv


class Rush:
    def __init__(self, name, id):
        self.name = name
        self.id = id
        self.cut = False
        self.scores = [0, 0, 0]
        self.interactions = [0, 0, 0]
        self.total_interactions = 0
        self.interaction_names = []
        self.db_score = 0
        self.comments = ""
        self.major = ""
        self.year = ""
        self.previously_knowns = ""
        self.clubs = ""
        self.gpa = 0.0
        self.interview_score = 0
        self.head_interviewer = ""
        self.pm_score = 0
        self.rush_score = 0
        self.rush_week_score = 0
        self.brotherhood_vote = ""

    def __repr__(self):
        return (
            self.name
            + " "
            + self.id
            + " "
            + str(self.cut)
            + " "
            + str(self.scores)
            + " "
            + str(self.interactions)
            + " "
            + str(self.db_score)
            + " "
            + str(self.interview_score)
            + " "
            + str(self.pm_score)
            + " "
            + str(self.rush_score)
            + " "
            + str(self.comments)
        )


# CHANGE THESE =================================================================================
rush_profile_filename = "real_data/rush_profile_responses_updated_3.csv"

sd_scores_filename = "real_data/sd_results_cleaned.csv"
mingle_scores_filename = "real_data/bm_results.csv"
bbq_scores_filename = "real_data/bbq_results.csv"

# no votes used since fa 22
# votes_filename = ["real_data/vote_responses_1.csv", "real_data/vote_responses_2.csv", "real_data/vote_responses_3.csv"]

interview_responses_filename = "real_data/interview_scores.csv"
rush_scores_filename = "real_data/rush_scores.csv"
pm_scores_filename = "real_data/pm_scores.csv"

cut_sheet_filename = "real_data/cut_sheet_cleaned.csv"
cut_sheet_2_filename = "real_data/cut_sheet_2.csv"

slides_info_filename = "real_data/slides_info.csv"  # output file
db_final_filename = "real_data/db_final_final.csv"  # output file
# ==============================================================================================

cut_sheets = [cut_sheet_filename, cut_sheet_2_filename]

score_filenames = [
    sd_scores_filename,
    mingle_scores_filename,
    bbq_scores_filename,
]

rushes = []

# rush profile data
with open(rush_profile_filename, newline="") as f:
    reader = csv.reader(f)
    data = list(reader)

for row in data[1:]:
    found = False
    for rush in rushes:
        if rush.id == row[2].lower():
            found = True
    if not found:
        new_rush = Rush(row[1], row[2].lower())
        new_rush.major = row[5]
        new_rush.year = row[8]
        new_rush.previously_knowns = row[10]
        new_rush.clubs = row[11]
        new_rush.gpa = row[12]
        new_rush.email = row[3]
        rushes.append(new_rush)

# event data
for i in range(3):
    # get csv of scores from an event
    with open(score_filenames[i], newline="") as f:
        reader = csv.reader(f)
        data = list(reader)
    # add scores to local data
    for row in data:
        for rush in rushes:
            if rush.id == row[1]:
                rush.scores[i] = float(row[2])
                rush.interactions[i] = int(row[4])
                rush.total_interactions += int(row[4])
                # try block just for testing
                try:
                    list_names = row[5].split(",")
                    for name in list_names:
                        if name.strip() not in rush.interaction_names:
                            rush.interaction_names.append(name.strip())
                except:
                    print('some error')
                    None
                if row[3] != "":
                    if rush.comments == "":
                        rush.comments = row[3]
                    else:
                        rush.comments = rush.comments + " | " + row[3]
                break

# tally up event scores
for rush in rushes:
    score_sum = 0
    interactions = 0
    for i in range(3):
        # database scores are weighted based on the number of interactions at each event
        score_sum += rush.scores[i] * rush.interactions[i]
        interactions += rush.interactions[i]
    if interactions > 0:
        rush.db_score = score_sum / interactions

# parse cuts
with open(cut_sheet_filename, newline="") as f:
    reader = csv.reader(f)
    data = list(reader)
for row in data:
    if row[7] == "T":
        id = row[1].lower()
        for rush in rushes:
            if rush.id == id:
                rush.cut = True
                break

# parse cuts 2
with open(cut_sheet_2_filename, newline="") as f:
    reader = csv.reader(f)
    data = list(reader)
for row in data:
    if row[10] == "T":
        id = row[1].lower()
        for rush in rushes:
            if rush.id == id:
                rush.cut = True
                break

# parse pm scores
with open(pm_scores_filename, newline="") as f:
    reader = csv.reader(f)
    data = list(reader)

for row in data[1:]:
    for rush in rushes:
        if rush.id == row[1]:
            if row[2] != '':
                rush.pm_score = float(row[2])
            break

# parse rush scores
with open(rush_scores_filename, newline="") as f:
    reader = csv.reader(f)
    data = list(reader)

for row in data[1:]:
    for rush in rushes:
        if rush.id == row[1]:
            rush.rush_score = float(row[2])
            break

# parse interview scores
with open(interview_responses_filename, newline="") as f:
    reader = csv.reader(f)
    data = list(reader)

for row in data[1:]:
    for rush in rushes:
        if rush.id == row[1]:
            if row[3] != '':
                rush.interview_score = float(row[3])
                rush.head_interviewer = row[2]
            break

# BROTHERHOOD VOTE REMOVED FA 22
'''
# parse brotherhood vote
for i in range(3):
    with open(votes_filename[i], newline="") as f:
        reader = csv.reader(f)
        data = list(reader)

    for col in range(2, len(data[0])):
        for row in range(len(data)):
            # id case
            if col % 2 == 0:
                if row == 0:
                    header = data[0][col]
                    split_data = header.split("/")
                    name = split_data[0]
                    id = split_data[1]
                    current_id = id
                    running_votes, running_total = 0, 0
            # votes case
            else:
                if data[row][col] == "Yes Bro :)":
                    running_votes += 1
                    running_total += 1
                elif data[row][col] == "No Bro :(":
                    running_total += 1
                if row == len(data) - 1:
                    for rush in rushes:
                        if rush.id == current_id:
                            rush.brotherhood_vote = (
                                str(running_votes) + "/" + str(running_total)
                            )
                            break
'''

# **************** THESE WEIGHTINGS CAN CHANGE SEMESTER TO SEMESTER ****************
for rush in rushes:
    rush.rush_week_score = (
        0.5 * rush.db_score
        + 0.03 * rush.interview_score  # extra 0 because interview scores are 0-10
        + 0.1 * rush.pm_score
        + 0.1 * rush.rush_score
    )

rushes.sort(key=lambda r: r.rush_week_score, reverse=True)

with open(slides_info_filename, "w") as csvfile:

    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(
        [
            "Name",
            "id",
            "Cut",
            "DB Score",
            "Interview",
            "Head Interviewer",
            "PM",
            "Rush",
            "Rush Week Score",
            "Major",
            "Year",
            "Previously Knowns",
            "Clubs",
            "GPA",
            "Unique Interactions",
        ]
    )

    for rush in rushes:
        if not rush.cut:
            csv_writer.writerow(
                [
                    rush.name,
                    rush.id,
                    rush.cut,
                    rush.db_score,
                    rush.interview_score,
                    rush.head_interviewer,
                    rush.pm_score,
                    rush.rush_score,
                    round(100 * rush.rush_week_score, 2),
                    rush.major,
                    rush.year,
                    rush.previously_knowns,
                    rush.clubs,
                    rush.gpa,
                    len(rush.interaction_names),
                ]
            )

with open(db_final_filename, "w") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(
        [
            "Name",
            "id",
            "Cut",
            "DB Score",
            "Interview",
            "Head Interviewer",
            "PM",
            "Rush",
            "Rush Week Score",
            "Major",
            "Year",
            "Previously Knowns",
            "Clubs",
            "GPA",
            "Unique Interactions",
            "Comments",
        ]
    )

    for rush in rushes:
        if not rush.cut:
            csv_writer.writerow(
                [
                    rush.name,
                    rush.id,
                    rush.cut,
                    rush.db_score,
                    rush.interview_score,
                    rush.head_interviewer,
                    rush.pm_score,
                    rush.rush_score,
                    rush.rush_week_score,
                    rush.major,
                    rush.year,
                    rush.previously_knowns,
                    rush.clubs,
                    rush.gpa,
                    len(rush.interaction_names),
                    rush.comments,
                ]
            )
